<a href="https://colab.research.google.com/github/see-3pO/Learning_Tensorflow/blob/master/Neural_Networks_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing necessary libraries and classes
import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras.datasets import mnist


In [ ]:
# Loading the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [ ]:
# Flatten the images
X_train = X_train.reshape(-1, 28*28).astype("float32") / 255.0
X_test = X_test.reshape(-1, 28*28).astype("float32") / 255.0

The Sequential API is convenient but not flexible when compared to the Functional API.

In [ ]:
# Creating a basic NN using Sequential API
model = keras.Sequential(
    [
        keras.Input(shape=(28*28)),
        layers.Dense(512, activation = 'relu'),
        layers.Dense(256, activation = 'relu'),
        layers.Dense(10),
    ]
)


In [ ]:
# Alternative method of creating the model
model = keras.Sequential()
model.add(keras.Input(shape=(28*28)))
model.add(layers.Dense(512, activation='relu'))
print(model.summary())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10))

In [ ]:
# get summary of network
model.summary()

In [ ]:
# compile
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy'],

)

# fit the model
model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=2)

# evaluate the model
model.evaluate(X_test, y_test, batch_size=32, verbose=2)

In [ ]:
# functional API model
from keras.models import Model

inputs = keras.Input(shape=(28*28))
x =  layers.Dense(512, activation='relu', name='first_layer')(inputs)
x = layers.Dense(265, activation='relu', name='second_layer')(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_8 (Dense)             (None, 512)               401920    
                                                                 
 dense_9 (Dense)             (None, 265)               135945    
                                                                 
 dense_10 (Dense)            (None, 10)                2660      
                                                                 
Total params: 540525 (2.06 MB)
Trainable params: 540525 (2.06 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# compile
# from logits= False since activation has been specified to softmax
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy'],
)

In [ ]:
# fit the model
model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=2)

# evaluate the model
model.evaluate(X_test, y_test, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 16s - loss: 0.1872 - accuracy: 0.9429 - 16s/epoch - 9ms/step
Epoch 2/5
1875/1875 - 14s - loss: 0.0799 - accuracy: 0.9754 - 14s/epoch - 8ms/step
Epoch 3/5
1875/1875 - 15s - loss: 0.0552 - accuracy: 0.9822 - 15s/epoch - 8ms/step
Epoch 4/5
1875/1875 - 15s - loss: 0.0398 - accuracy: 0.9874 - 15s/epoch - 8ms/step
Epoch 5/5
1875/1875 - 17s - loss: 0.0330 - accuracy: 0.9897 - 17s/epoch - 9ms/step
313/313 - 1s - loss: 0.0820 - accuracy: 0.9780 - 1s/epoch - 3ms/step


[0.08195251226425171, 0.9779999852180481]

#### Extracting specific layer features


In [ ]:
# Say you have this neural network model
model = keras.Sequential()
model.add(keras.Input(shape=(28*28)))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu', name='feature_layer'))
model.add(layers.Dense(10))

# -2 gives us the second last layer of model
nn_model = Model(inputs=model.inputs,
                 outputs=[model.layers[-2].output])

# using the name to specify output
nn_model_two = Model(inputs= model.inputs,
                     outputs=model.get_layer('feature_layer').output)

feature = nn_model.predict(X_train)
print(feature.shape)

1875/1875 [==============================] - 6s 3ms/step
(60000, 256)


In [ ]:
feature = nn_model_two.predict(X_train)
print(feature.shape)

1875/1875 [==============================] - 5s 3ms/step
(60000, 256)


In [ ]:
# obtaining output for all the layers
nn_model_three = Model(inputs=model.inputs,
                       outputs=[layer.output for layer in model.layers])


In [ ]:
features = nn_model_three.predict(X_train)
for feature in features:
    print(feature.shape)

1875/1875 [==============================] - 7s 4ms/step
(60000, 512)
(60000, 256)
(60000, 10)
